In [1]:
# useing this variable for saving models and summaries
exp_name = "CNN-LSTM4002"

## Running CNN then RNN
5 layers CNN [2, 2, 2, 2, 2], then RNN, bias_shape = [32,64,64, 64, 64]

In [4]:
%load_ext autoreload
%autoreload 2
import numpy as np
import tensorflow as tf
import sys
from hdf5storage import savemat
import pickle

sys.path.append('../../../src')
sys.path.append('../../../src/models')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import models.CNN_lstm_model as CNN_lstm_model
import training  
import evaluation.nn_eval as nn_eval 

## Loading synthetic data

In [6]:
#loading data
data_path= '/data2/data/zebrahim/synthetic_dataset/data_8192_1000_3_4_4_.03_.03_.2' + 'processed_data.p'

with open(data_path, 'r') as fin:
    data = pickle.load(fin)
    
train_data = data['train_data']
validation_data = data['validation_data']
test_data = data['test_data']

train_gt = np.expand_dims(data['train_gt'], axis=-1)
validation_gt = np.expand_dims(data['validation_gt'], axis=-1)
test_gt = np.expand_dims(data['test_gt'], axis=-1)

index_of_changes = data['index_of_changes']
 

## Model

In [7]:
## Model Parameter
n_variables= 12
learning_rate = 0.001
batch_size = 10

pooling_stride = [32, 1, 1, 1, 1, 1]
weights_shape = [[7,n_variables, 32],
                 [7, 32, 32],
                 [7, 32, 32],
                 [7, 32, 32],
                 [7, 32, 32],
                 [7, 32, 32]]

bias_shape = [32, 32, 32, 32, 32, 32, 32]
activation = tf.nn.relu

lstm_n_hidden = 128
n_classes = 1


#positive weight for weighted cross entropy
s = train_data.shape
resolution = np.prod(pooling_stride)
gt_effective = np.reshape(train_gt, (s[0]*(s[1]/resolution), resolution))
pos_weight = (1.0 - gt_effective.max(axis=1).mean()) / gt_effective.max(axis=1).mean()

In [8]:
pos_weight

3.186018221237171

In [9]:
# Launch the graph
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [10]:
summary_path = '../summary/' + exp_name 
checkpoint_path = '../model/' + exp_name + '/' + exp_name 


In [11]:
model = CNN_lstm_model.Model(n_variables, n_classes)

In [12]:
model.build_cnn_lstm_net(pooling_stride,
                         weights_shape,
                         bias_shape,
                         activation,
                         lstm_n_hidden,
                         n_classes,
                         learning_rate,
                         pos_weight)

In [13]:
train_obj = training.Trainer(model,
                 train_data, train_gt,
                 validation_data, validation_gt,
                 sess,
                 summary_path,
                 batch_size,
                 global_step = 0)

In [14]:
train_obj.train(max_iter = 10000,
                train_eval_step=100,
                validation_eval_step=100,
                display_step=100)

Tarin Iter 0, auc= 0.242593, fmeasure_max= 0.382295, precision= 0.244612, recall= 0.874552
Train Random guess:  , auc= 0.218718fmeasure_max= 0.359640, precision= 0.220859, recall= 0.967742
Validation Iter 0, auc= 0.244315, fmeasure_max= 0.393723, precision= 0.272368, recall= 0.710120
Validation Random guess:  auc= 0.234920, fmeasure_max= 0.370983, precision= 0.227734, recall= 1.000000
Minibatch Loss= 1.019436
Tarin Iter 100, auc= 0.680983, fmeasure_max= 0.709887, precision= 0.640351, recall= 0.796364
Train Random guess:  , auc= 0.226962fmeasure_max= 0.355380, precision= 0.216955, recall= 0.981818
Validation Iter 100, auc= 0.624900, fmeasure_max= 0.656452, precision= 0.619482, recall= 0.698113
Validation Random guess:  auc= 0.233961, fmeasure_max= 0.372197, precision= 0.228830, recall= 0.996569
Minibatch Loss= 0.755546
Tarin Iter 200, auc= 0.791015, fmeasure_max= 0.739382, precision= 0.730916, recall= 0.748047
Train Random guess:  , auc= 0.202390fmeasure_max= 0.334555, precision= 0.2016

KeyboardInterrupt: 

In [13]:
saver = tf.train.Saver()
saver.save(sess, checkpoint_path, global_step=train_obj.global_step)

'../model/CNN-LSTM4001/CNN-LSTM4001-10000'

In [ ]:
saver = tf.train.Saver()
saver.restore(sess, checkpoint_path+'-1000')

In [15]:
probability_of_pos = nn_eval.evaluate_model(model,
                                   sess,
                                   test_data,
                                   test_gt)

Testing auc = 0.924960 F1_max = 0.856121 prec = 0.885400 rec = 0.828716
Test Random guess: auc= 0.229861 fmeasure_max= 0.375674 , precision= 0.231298 , recall= 0.999662


In [15]:
#shape of probability_change is (batch size, T, num_classes)
#since num_classes = 1 we squeeze that axis
probability_change = np.squeeze(probability_of_pos, axis = 2)

In [16]:
savemat('../poc/poc'+exp_name+'.mat', {'poc':probability_change})